<a href="https://colab.research.google.com/github/Rutviag/AI-ML-IISc-project---Group-11/blob/main/New_file_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1) Reading all files and just combining complete data

In [ ]:
# Import necessary libraries
import pandas as pd
import os

# Step 1: Define the directory containing the uploaded files
# In Colab, uploaded files are in the current working directory
uploaded_files_dir = '.'  # Adjust this if you have a specific directory for your files

# Step 2: Get all CSV files in the directory
csv_files = [file for file in os.listdir(uploaded_files_dir) if file.endswith('.csv')]

# Step 3: Combine all CSV files
dataframes = []
for file_name in csv_files:
    print(f"Reading {file_name}...")
    df = pd.read_csv(file_name)
    dataframes.append(df)

# Combine all DataFrames into one
consolidated_data = pd.concat(dataframes, ignore_index=True)

# Step 4: Save the consolidated dataset
output_file = 'consolidated_data.csv'
consolidated_data.to_csv(output_file, index=False)
print(f"Consolidation complete. File saved as {output_file} in Colab.")


Reading 11 Nov.csv...
Reading 08 Aug.csv...
Reading 10 Oct.csv...
Reading 04 April.csv...
Reading 07 July.csv...
Reading 12 Dec.csv...
Reading 06 June.csv...
Reading 01 Jan.csv...
Reading 05 May.csv...
Reading 09 Nov.csv...
Reading 03 March.csv...
Reading 02 Feb.csv...
Consolidation complete. File saved as consolidated_data.csv in Colab.


2) Getting number of line items

In [ ]:
# Read the consolidated CSV file (if not already loaded)
df = pd.read_csv('consolidated_data.csv')  # Replace with your file name if different

# Get the number of rows (line items)
line_items_count = len(df)

print(f"The number of line items in the dataset is: {line_items_count}")

The number of line items in the dataset is: 1961484


3) Removing extra coumns

In [ ]:
# Read the consolidated CSV file
df = pd.read_csv('consolidated_data.csv')  # Replace with your file name if different

# Drop the specified columns
columns_to_remove = ['YEAR', 'DEP_TIME', 'FLIGHTS']
df_cleaned = df.drop(columns=columns_to_remove, errors='ignore')

# Save the cleaned dataset
output_file = 'cleaned_data.csv'
df_cleaned.to_csv(output_file, index=False)

print(f"Columns {columns_to_remove} removed. Cleaned data saved as {output_file}.")


Columns ['YEAR', 'DEP_TIME', 'FLIGHTS'] removed. Cleaned data saved as cleaned_data.csv.


4.1 Step 4: Selecting Top 10 Cities

In [ ]:
# Step 4: Selecting top 10 cities
file_path = 'cleaned_data.csv'  # Replace with your file name
df = pd.read_csv(file_path)

# Print the number of rows in the dataset
num_rows = df.shape[0]
print(f"Number of rows in the dataset: {num_rows}")

# Get the top 10 origin cities by flight count
top_cities = df['ORIGIN_CITY_NAME'].value_counts().head(10)

# Display the top 10 cities
print("\nTop 10 Origin Cities Based on Number of Flights:")
print(top_cities)

# Filter the dataset to include only the top 10 origin cities
top_cities_list = top_cities.index.tolist()
df_filtered = df[df['ORIGIN_CITY_NAME'].isin(top_cities_list)]

# Save the filtered dataset
output_file = 'filtered_data_top_10_cities.csv'
df_filtered.to_csv(output_file, index=False)
print(f"\nFiltered data with top 10 cities saved as {output_file}.")

file_path = 'filtered_data_top_10_cities.csv'  # Replace with your file name
df = pd.read_csv(file_path)

# Print the number of rows in the dataset
num_rows = df.shape[0]
print(f"Number of rows in the dataset: {num_rows}")


Number of rows in the dataset: 1961484

Top 10 Origin Cities Based on Number of Flights:
ORIGIN_CITY_NAME
New York, NY             299738
Chicago, IL               92985
Atlanta, GA               74370
Dallas/Fort Worth, TX     64575
Denver, CO                62243
Washington, DC            57072
Charlotte, NC             55772
Houston, TX               45547
Los Angeles, CA           42980
Orlando, FL               40981
Name: count, dtype: int64

Filtered data with top 10 cities saved as filtered_data_top_10_cities.csv.
Number of rows in the dataset: 836263


5) Cleaning the Date Format

In [ ]:
# Step 5: Cleaning and extracting date from FL_DATE
file_path = 'filtered_data_top_10_cities.csv'  # Replace with your file name
df = pd.read_csv(file_path)

# Ensure FL_DATE is treated as a string
df['FL_DATE'] = df['FL_DATE'].astype(str)

# Normalize the formats in FL_DATE
# Handle MM-DD-YYYY and MM/DD/YYYY formats
df['FL_DATE'] = df['FL_DATE'].str.extract(r'(\d{2}-\d{2}-\d{4})|(\d{2}/\d{2}/\d{4})')[0].fillna(
    df['FL_DATE'].str.extract(r'(\d{2}-\d{2}-\d{4})|(\d{2}/\d{2}/\d{4})')[1]
)

# Convert FL_DATE to datetime
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'], format='%m-%d-%Y', errors='coerce').fillna(
    pd.to_datetime(df['FL_DATE'], format='%m/%d/%Y', errors='coerce')
)

# Handle invalid dates
df['FL_DATE'] = df['FL_DATE'].fillna(pd.Timestamp('1900-01-01'))  # Default for invalid dates

# Extract Month and Date
df['Month'] = df['FL_DATE'].dt.month
df['Date'] = df['FL_DATE'].dt.day

# Drop the original FL_DATE column
df = df.drop(columns=['FL_DATE'])

# Save the updated dataset
output_file = 'updated_data_with_cleaned_date.csv'
df.to_csv(output_file, index=False)
print(f"'Month' and 'Date' extracted and 'FL_DATE' cleaned. Updated data saved as {output_file}.")


'Month' and 'Date' extracted and 'FL_DATE' cleaned. Updated data saved as updated_data_with_cleaned_date.csv.


6) Weather data conversion

In [ ]:
file_path = 'updated_data_with_cleaned_date.csv'  # Replace with the file name if needed
df = pd.read_csv(file_path)

# Print the number of rows in the filtered dataset
num_rows = df.shape[0]
print(f"Number of rows in the filtered dataset: {num_rows}")

# Define thresholds for weather delay categories
def categorize_weather_delay(delay):
    if pd.isna(delay) or delay == 0:  # No delay or missing data
        return 'favorable'
    elif delay <= 30:  # Minor delay
        return 'risky'
    else:  # Significant delay
        return 'unfavorable'

# Convert `WEATHER_DELAY` to categories
df['Weather_Category'] = df['WEATHER_DELAY'].apply(categorize_weather_delay)

# Remove the original `WEATHER_DELAY` column
df = df.drop(columns=['WEATHER_DELAY'])

# Save the updated dataset
output_file = 'filtered_data_after_weather_update.csv'
df.to_csv(output_file, index=False)

print(f"\nWeather delay categorized and original `WEATHER_DELAY` column removed. Updated data saved as {output_file}.")


Number of rows in the filtered dataset: 836263

Weather delay categorized and original `WEATHER_DELAY` column removed. Updated data saved as filtered_data_after_weather_update.csv.


7) NAS delay converison

In [ ]:
file_path = 'filtered_data_after_weather_update.csv'  # Replace with the file name if needed
df = pd.read_csv(file_path)

# Print the number of rows in the dataset
num_rows = df.shape[0]
print(f"Number of rows in the dataset: {num_rows}")

# Define thresholds for NAS delay categories
def categorize_nas_delay(delay):
    if pd.isna(delay) or delay == 0:  # No delay or missing data
        return 'favorable'
    elif delay <= 30:  # Minor delay
        return 'risky'
    else:  # Significant delay
        return 'unfavorable'

# Convert `NAS_DELAY` to categories
df['NAS_Category'] = df['NAS_DELAY'].apply(categorize_nas_delay)

# Remove the original `NAS_DELAY` column
df = df.drop(columns=['NAS_DELAY'])

# Save the updated dataset
output_file = 'filtered_data_after_nas_update.csv'
df.to_csv(output_file, index=False)

print(f"\nNAS delay categorized and original `NAS_DELAY` column removed. Updated data saved as {output_file}.")


Number of rows in the dataset: 836263

NAS delay categorized and original `NAS_DELAY` column removed. Updated data saved as filtered_data_after_nas_update.csv.


8) Categorize the Carrier as reliable or unreliable , remove "LATE_AIRCRAFT_DELAY" and "Year"columns

In [ ]:
import pandas as pd
# Load the dataset after NAS update
file_path = 'filtered_data_after_nas_update.csv'  # Replace with the file name if needed
df = pd.read_csv(file_path)

# Print the number of rows in the dataset
num_rows = df.shape[0]
print(f"Number of rows in the dataset: {num_rows}")

# Step 1: Calculate the historical average delay for each carrier
carrier_avg_delay = df.groupby('OP_CARRIER_FL_NUM')['CARRIER_DELAY'].mean()

# Step 2: Map historical average delay back to the original DataFrame
df['Carrier_Avg_Delay'] = df['OP_CARRIER_FL_NUM'].map(carrier_avg_delay)

# Step 3: Categorize delays based on historical averages
def categorize_carrier_delay(row):
    if pd.isna(row['CARRIER_DELAY']) or row['CARRIER_DELAY'] <= row['Carrier_Avg_Delay']:
        return 'reliable'
    else:
        return 'unreliable'

df['Carrier_Category'] = df.apply(categorize_carrier_delay, axis=1)

# Step 4: Remove the `CARRIER_DELAY`, `LATE_AIRCRAFT_DELAY` columns
df = df.drop(columns=['CARRIER_DELAY', 'Carrier_Avg_Delay', 'LATE_AIRCRAFT_DELAY', ])

# Save the updated dataset
output_file = 'filtered_data_after_carrier_update.csv'
df.to_csv(output_file, index=False)

print(f"\nCarrier delay categorized as `reliable` or `unreliable`, and `CARRIER_DELAY`, and `LATE_AIRCRAFT_DELAY` columns removed.")
print(f"Updated data saved as {output_file}.")


Number of rows in the dataset: 836263

Carrier delay categorized as `reliable` or `unreliable`, and `CARRIER_DELAY`, `LATE_AIRCRAFT_DELAY`, `YEAR`, and `FLIGHTS` columns removed.
Updated data saved as filtered_data_after_carrier_update.csv.


Update codes from here

In [17]:
import pandas as pd
import numpy as np

# Read the CSV file
file_path = 'filtered_data_after_carrier_update.csv'  # Replace with your correct file path
flight_data = pd.read_csv(file_path)

# Convert DEP_DELAY to non-negative values
flight_data['DEP_DELAY'] = np.clip(flight_data['DEP_DELAY'], a_min=0, a_max=None)

# Save the updated DataFrame back to the same CSV file
flight_data.to_csv(file_path, index=False)

print(f"The file '{file_path}' has been updated with non-negative DEP_DELAY values.")


The file 'filtered_data_after_carrier_update.csv' has been updated with non-negative DEP_DELAY values.


1) Upload and Read the Dataset

In [18]:
# Import necessary libraries
import pandas as pd
import time

# Install necessary libraries
!pip install catboost xgboost --quiet

# Import models
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score


# Load the dataset
file_path = 'filtered_data_after_carrier_update.csv'  # Update the file path as needed
data = pd.read_csv(file_path)

# Display basic information
print("Dataset Information:")
print(data.info())

# Preview the data
print("\nDataset Preview:")
print(data.head())


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 836263 entries, 0 to 836262
Data columns (total 17 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   DAY_OF_WEEK        836263 non-null  int64  
 1   OP_UNIQUE_CARRIER  836263 non-null  object 
 2   TAIL_NUM           833184 non-null  object 
 3   OP_CARRIER_FL_NUM  836263 non-null  int64  
 4   ORIGIN_AIRPORT_ID  836263 non-null  int64  
 5   ORIGIN_CITY_NAME   836263 non-null  object 
 6   DEST_AIRPORT_ID    836263 non-null  int64  
 7   DEST_CITY_NAME     836263 non-null  object 
 8   CRS_DEP_TIME       836263 non-null  int64  
 9   DEP_DELAY          820003 non-null  float64
 10  AIR_TIME           678641 non-null  float64
 11  DISTANCE           695221 non-null  float64
 12  Month              836263 non-null  int64  
 13  Date               836263 non-null  int64  
 14  Weather_Category   836263 non-null  object 
 15  NAS_Category       836263 non-

2) Data Cleaning

In [19]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder


# Drop `TAIL_NUM` and handle missing values
data = data.drop(columns=['TAIL_NUM'], errors='ignore')
data = data.dropna(subset=['DEP_DELAY'])  # Drop rows with missing target variable
columns_to_impute = ['AIR_TIME', 'DISTANCE']
imputer = SimpleImputer(strategy='median')
data[columns_to_impute] = imputer.fit_transform(data[columns_to_impute])

# Encode categorical features
object_columns = data.select_dtypes(include=['object']).columns
print(f"Object Columns to Encode: {object_columns}")

label_encoders = {}
for column in object_columns:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column].astype(str))
    label_encoders[column] = le

print("\nData After Encoding:")
print(data.info())


Object Columns to Encode: Index(['OP_UNIQUE_CARRIER', 'ORIGIN_CITY_NAME', 'DEST_CITY_NAME',
       'Weather_Category', 'NAS_Category', 'Carrier_Category'],
      dtype='object')

Data After Encoding:
<class 'pandas.core.frame.DataFrame'>
Index: 820003 entries, 0 to 836262
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   DAY_OF_WEEK        820003 non-null  int64  
 1   OP_UNIQUE_CARRIER  820003 non-null  int64  
 2   OP_CARRIER_FL_NUM  820003 non-null  int64  
 3   ORIGIN_AIRPORT_ID  820003 non-null  int64  
 4   ORIGIN_CITY_NAME   820003 non-null  int64  
 5   DEST_AIRPORT_ID    820003 non-null  int64  
 6   DEST_CITY_NAME     820003 non-null  int64  
 7   CRS_DEP_TIME       820003 non-null  int64  
 8   DEP_DELAY          820003 non-null  float64
 9   AIR_TIME           820003 non-null  float64
 10  DISTANCE           820003 non-null  float64
 11  Month              820003 non-null  int64  
 12  Dat

Step 3: Feature Engineering

In [20]:
# Extract hour from CRS_DEP_TIME (scheduled departure time)
data['DEP_HOUR'] = (data['CRS_DEP_TIME'] // 100).astype(int)

# Preview to confirm the DEP_HOUR column
print(data[['CRS_DEP_TIME', 'DEP_HOUR']].head())



   CRS_DEP_TIME  DEP_HOUR
0          1029        10
1          1629        16
2          1906        19
3          1559        15
4           900         9


In [21]:
# Categorical buckets for DEP_HOUR
def categorize_dep_hour(hour):
    if 0 <= hour < 6:
        return "Night"
    elif 6 <= hour < 12:
        return "Morning"
    elif 12 <= hour < 18:
        return "Afternoon"
    else:
        return "Evening"

# Add DEP_HOUR_CATEGORY and one-hot encode it
data['DEP_HOUR_CATEGORY'] = data['DEP_HOUR'].apply(categorize_dep_hour)
data = pd.get_dummies(data, columns=['DEP_HOUR_CATEGORY'], drop_first=True)

# Add peak hour feature
def is_peak_hour(hour):
    return 1 if (7 <= hour <= 10 or 16 <= hour <= 19) else 0

data['IS_PEAK_HOUR'] = data['DEP_HOUR'].apply(is_peak_hour)

# Add minutes since midnight
data['MINUTES_SINCE_MIDNIGHT'] = data['DEP_HOUR'] * 60

# Add interaction features
data['DEP_HOUR_DAY_OF_WEEK'] = data['DEP_HOUR'] * data['DAY_OF_WEEK']
data['DEP_HOUR_DISTANCE'] = data['DEP_HOUR'] * data['DISTANCE']

# Preview updated dataset
print("Feature Engineering Complete. Updated Dataset:")
print(data.head())


Feature Engineering Complete. Updated Dataset:
   DAY_OF_WEEK  OP_UNIQUE_CARRIER  OP_CARRIER_FL_NUM  ORIGIN_AIRPORT_ID  \
0            1                  0               4975              12953   
1            1                  0               5163              12953   
2            1                  0               5110              11057   
3            1                  0               5199              12953   
4            1                  0               4908              12953   

   ORIGIN_CITY_NAME  DEST_AIRPORT_ID  DEST_CITY_NAME  CRS_DEP_TIME  DEP_DELAY  \
0                 7            10994              43          1029        0.0   
1                 7            11042              53          1629        0.0   
2                 1            12478             192          1906        0.0   
3                 7            11057              46          1559        0.0   
4                 7            14685             248           900        9.0   

   AIR_TIME  ..

Step 4: Scale features

In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA


# Standardize the features (PCA works better on standardized data)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA
pca = PCA(n_components=0.95)  # Retain 95% of the variance
X_pca = pca.fit_transform(X_scaled)

# Print explained variance ratio to understand how much variance is retained
print("Explained variance ratio by PCA components:", pca.explained_variance_ratio_)
print("Number of components selected:", pca.n_components_)

# Define features (X) and target variable (y)
X = data.drop(columns=['DEP_DELAY'], errors='ignore')  # Features
y = data['DEP_DELAY']                                  # Target variable

# Initialize the scaler
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# Split into train-test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

# Confirm no missing values in the target variable
print(f"Missing values in y_train: {y_train.isna().sum()}")
print(f"Missing values in y_test: {y_test.isna().sum()}")


Missing values in y_train: 0
Missing values in y_test: 0


Step 5: Train Traditional Models

In [32]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score


# Install required libraries
!pip install catboost xgboost --quiet


# Define models
models = {
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "CatBoost": CatBoostRegressor(iterations=800, learning_rate=0.1, depth=10,verbose=0),
    "XGBoost": XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
}

# Function to calculate accuracy
def calculate_accuracy(y_true, y_pred, threshold=20):  # Threshold = ±20 minutes
    correct_predictions = abs(y_true - y_pred) <= threshold
    return (correct_predictions.sum() / len(y_true)) * 100

#stat=time.time()
# Train and evaluate models
results = {}

for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Evaluate performance
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    accuracy = calculate_accuracy(y_test, y_pred, threshold=20)  # Calculate accuracy

    results[name] = {"MSE": mse, "R2": r2, "Accuracy (%)": accuracy}

#print("Time taken for model": )

# Convert results to DataFrame
results_df = pd.DataFrame(results).T
print("\nModel Performance:")
print(results_df)




Model Performance:
                  MSE        R2  Accuracy (%)
CatBoost  2426.967099  0.307753     77.432206
XGBoost   2453.242116  0.300259     77.121231


Select Best Model